In [ ]:
import pandas as pd
import os
import numpy as np
import time
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
import keras
from keras import layers
import pickle

## Hyper parameter tuning

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/60+0_7_full/new_0_9.csv")

In [ ]:
from keras import models
from keras import optimizers

df_org = df
history_list = []
test_loss = []
test_y = []

In [ ]:
test_threshold = [20]
test_movenum = [60, 30, 20]
for threshold in test_threshold:
  df = df_org
  df.loc[df.Eval >= threshold, 'Eval'] = threshold
  df.loc[df.Eval <= -threshold, 'Eval'] = -threshold
  df.loc[df.EvalDiff >= threshold, 'Eval'] = threshold
  df.loc[df.EvalDiff <= -threshold, 'Eval'] = -threshold
  df['Time'] = df['Time'] / 60
  df['Eval'] = df['Eval'] / threshold
  df['EvalDiff']  = df['EvalDiff'] / threshold

  df = df[['GameIndex', 'Time', 'Eval', 'EvalDiff', 'ForcedMate',
            'WhiteElo', 'BlackElo', 'Result']]
  gindex = list(df["GameIndex"].unique())
  df_output = pd.DataFrame(gindex)
  df_output.columns = ['GameIndex']
  first_ = 1
  tb_list = []
  movenum_max = test_movenum[0] * 2
  for g in gindex:
      df_temp = df[df.GameIndex == g]
      df_output.loc[df_output.GameIndex == g, 'WhiteElo'] = df_temp['WhiteElo'].iloc[0]
      df_output.loc[df_output.GameIndex == g, 'BlackElo'] = df_temp['BlackElo'].iloc[0]
      df_output.loc[df_output.GameIndex == g, 'Result'] = df_temp['Result'].iloc[0]
      table = df_temp[['Eval', 'EvalDiff', 'Time']].transpose()
      table.columns = [str(x) for x in range(0, len(table.iloc[0]))]
      if(len(table.iloc[0]) > movenum_max):
        table = table[[str(x) for x in range(0, movenum_max)]]
      else:
        newtb = pd.DataFrame(columns = [str(x) for x in range(len(table.iloc[0]), movenum_max)])
        table = pd.concat([table, newtb], axis=1)
        table.columns = [str(x) for x in range(0, len(table.iloc[0]))]
        table = table.fillna(0)
      tb_list.append(table)
  df_output['avg_elo'] = (df_output['WhiteElo'] + df_output['BlackElo']) / 2

  for movenum_max in test_movenum:
    column_list = [str(x) for x in range(0, movenum_max)]
    if(movenum_max != test_movenum[0]):
      tb_list = [tb[column_list] for tb in tb_list]

    X_train, X_test, y_train, y_test = train_test_split(tb_list, df_output[['avg_elo']], test_size=0.2, random_state=42)

    model = models.Sequential()
    model.add(layers.Conv2D(64, (5, 5), padding = 'same', activation='relu', input_shape=(3, movenum_max, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.ZeroPadding2D(padding = (2, 2), data_format = 'channels_last'))

    model.add(layers.Conv2D(64, (3, 3), padding = 'same', activation='relu', input_shape=(3, movenum_max, 1)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    model.add(layers.Dense(1, activation = 'relu'))


    X_train = np.array(X_train)
    X_train = tf.reshape(X_train, shape = [len(X_train), 3, movenum_max, 1])
    model.compile(optimizer= optimizers.Adam(),
                  loss=tf.keras.losses.mse)
    history = model.fit(X_train, y_train, epochs= 10)
    X_test = np.array(X_test)
    X_test = tf.reshape(X_test, shape = [len(y_test), 3, movenum_max, 1])
    y_predict = model.predict(X_test)
    score = model.evaluate(X_test, y_test)

    history_list.append(history.history)
    test_loss.append(score)
    test_y.append(y_predict)

In [ ]:
df_vis = pd.DataFrame(y_predict)
df_vis.columns = ['avg_elo_pred']
df_vis['avg_elo_pred'] = df_vis['avg_elo_pred'] * 3500
y_test_add = y_test * 3500
df_vis = df_vis.reset_index()
df_vis = pd.concat([df_vis, y_test_add.reset_index()], axis = 1)

In [ ]:
import seaborn as sns
sns.scatterplot(data = df_vis, x = 'avg_elo_pred', y = 'avg_elo', alpha = 0.2)